In [12]:
import os

import conf
from dotenv import load_dotenv
from langchain.prompts import (
    PipelinePromptTemplate,
    PromptTemplate,
)
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import OpenAI
from langchain.chains import LLMChain

In [13]:
load_dotenv()
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

llm = OpenAI(openai_api_key=OPENAI_API_KEY)
full_template = conf.PROMPTS["FULL_TEMPLATE"]
role_template = conf.PROMPTS["ROLE_TEMPLATE"]
instruction_template = conf.PROMPTS["INSTRUCTION_TEMPLATE"]
example_template = conf.PROMPTS["EXAMPLE_TEMPLATE"]
start_template = conf.PROMPTS["START_TEMPLATE"]

full_prompt = PromptTemplate.from_template(full_template)
role_prompt = PromptTemplate.from_template(role_template)
instruction_prompt = PromptTemplate.from_template(instruction_template)
example_prompt = PromptTemplate.from_template(example_template)
start_prompt = PromptTemplate.from_template(start_template)

input_prompts = [
    ("role", role_prompt),
    ("instruction", instruction_prompt),
    ("example", example_prompt),
    ("start", start_prompt),
]
pipeline_prompt = PipelinePromptTemplate(final_prompt=full_prompt, pipeline_prompts=input_prompts)

In [16]:
user_instruction = "Standardize the phone numbers. See examples below"
user_example = "(001658345423, +491658345423), (01743456732, +491743456732), (1845342361, +491845342361), (1743567891, +491743567891)"

In [20]:
user_instruction = "Standardize dates to dd.mm.yyyy format"
# user_example = "(13.01.91, 13.01.1991), (02.27.2016, 27.02.2016), (1994-12-13, 13.12.1994) "
user_example = ""

In [22]:
user_instruction = "Standardize address to the following format: {street} {house number}, {post index} city"
# user_example = "(13.01.91, 13.01.1991), (02.27.2016, 27.02.2016), (1994-12-13, 13.12.1994) "
user_example = "(20095 Hamburg/Rathausmarkt 1, Rathausmarkt 1, 20095 Hamburg)"

In [23]:
llm_chain = pipeline_prompt | llm
llm_chain.invoke({"user_example": user_example, "user_instruction": user_instruction})

' address: (values) => { values.map(([item, index]) => { let address = item; let houseNumber = \'\'; let postIndex = \'\'; let city = \'\'; //split address into street and house number if (/\\d/.test(address)) { const numIndex = address.search(/\\d/); houseNumber = address.slice(numIndex, address.length); address = address.slice(0, numIndex).trim(); } //split address by comma if (address.includes(\',\')) { const splitAddress = address.split(\',\'); //check if last element is a number if (/\\d/.test(splitAddress[splitAddress.length - 1].trim())) { postIndex = splitAddress[splitAddress.length - 1].trim(); city = splitAddress[splitAddress.length - 2].trim(); } else { city = splitAddress[splitAddress.length - 1].trim(); } } else { city = address; } //combine all elements into standardized address format address = `${address} ${houseNumber}, ${postIndex} ${city}`; return [{ value: address, info: [{ message: \'We have automatically standardized your address to the correct format.\', level: "